# Задача

Главные задачи, которые должен выполнять скрипт:
+ Извлекать реплики с приветствием – где менеджер поздоровался. 
+ Извлекать реплики, где менеджер представил себя. 
+ Извлекать имя менеджера. 
+ Извлекать название компании. 
+ Извлекать реплики, где менеджер попрощался.
+ Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


---
Извлечение имя менеджера:
   + Прямое - менеджер назвал свое имя
   + Косвенное - клиент обратился к менеджеру по имени

## Загрузка библиотек

In [1]:
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
import pymorphy2

from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline
from yargy.predicates import gram
from yargy.interpretation import fact

nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

## Вспомогательные функции

In [2]:
def generate_regex(list_examples: list):
    """
    Создание регулярных выражений для каждой строки из списка
    """
    list_regex = []
    for string in list_examples:
        result = r'(?:^|(\w+)\s)'
        if ' ' in string:
            lst = string.split()
            for i in range(len(lst)):
                result += lst[i]
                if i != (len(lst)-1):
                    result += '\s?(.*?)\s'                    
        else:
            result += string
            
        result += '(?:$|\s(\w+))'
        list_regex.append(result)
    return list_regex 
 
def is_indicator(text: str, indicators: list):
    """
    Проверка наличия подстроки (из списка) в строке. 
    """
    list_regex = generate_regex(indicators)
    regex = re.compile(r'|'.join(list_regex))
    result = re.search(regex, text.lower().strip())
    return bool(result)
       
def company_recognize(text: str, parser):
    """
    Извлечение названия компании. 
    """
    companies = [match.fact.name for match in parser.findall(text.lower())]
    companies = [company for company in companies if company not in russian_stopwords]
    return ', '.join(companies) if companies else None

def name_recognize(text:str, parser):
    """
    Извлечение имени 
    """
    names = [match.fact.name for match in parser.findall(text.lower())]
    # (Неоднозначность морфологической формы слова в yargy) поэтому дополнительная проверка
    names = [name for name in names if 'Name' in morph.parse(name)[0].tag]
    return ', '.join(names) if names else None
          
def role_name(row: pd.Series, role, parser_direct, parser_indirect):
    """
    Извлечение имени (менеджера). 
    """
    # менеджер сам представился (предпочтительнее)
    if row['role'] == role:   
        return name_recognize(row['text'], parser_direct)

    # косвенное обращение к менеджеру
    else:
        return name_recognize(row['text'], parser_indirect)

def polite_person(row: pd.Series):
    """
    Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»
    """
    flag = False
    if row['greeting'] and row['goodbye']:
        flag = True
    return flag

def extract_values(series: pd.Series):
    """
    Обединение значений из разных ячеек в одну строку
    """
    values = list(set(series[series.notna()]))
    return  ', '.join(values) if values else None

## Инициализация переменных

In [3]:
GREETINGS = ['добрый день', 'добрый вечер', 'доброе утро', 'здравствуйте', 'привет']
GOODBYES = ['до свидания', 'хорошего вечера', 'хорошего дня', 'всего хорошего', 'всего доброго']
DIRECT_INDICATORS = ['это', 'зовут меня', 'меня зовут']
REPRESENTATIONS = ['я представитель', 'представляю интересы', 'являюсь представителем', 'меня зовут', 'зовут меня']
ORGANIZATIONS = ['компания', 'организация', 'фирма', 'корпорация']

morph = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['который', 'ага', 'угу', 'алло', 'сами'])

# грамеры
NOUN = gram('NOUN') # сущ
NAME = gram('Name') # имя
ADJF = gram('ADJF') # прил
PER_2 = gram('2per') # 2 лицо
IMPR = gram('impr') # повелительное наклонение


# КОМПАНИЯ
Company = fact(
    'Company',
    ['name']
)
INDICATORS = morph_pipeline(ORGANIZATIONS)
COMPANY_NAME = rule(ADJF.optional(), NOUN.repeatable().interpretation(Company.name))
ORGANIZATION = rule(INDICATORS, COMPANY_NAME
                   ).interpretation(Company)
parser_company = Parser(ORGANIZATION)

# ИМЯ 
Person = fact(
    'Person',
    ['name']
)
NAME = NAME.interpretation(Person.name)
GREETING = morph_pipeline(GREETINGS)
GOODBYE = morph_pipeline(GOODBYES)

# (обращение к персоне)
PERSON_INDIRECT = or_(rule(or_(PER_2, IMPR), NAME),
                      rule(NAME, or_(PER_2, IMPR)),
                      rule(or_(GREETING, GOODBYE), NAME),
                      rule(NAME, or_(GREETING, GOODBYE))
                     ).interpretation(Person)
parser_person_indirect = Parser(PERSON_INDIRECT)

# (представился сам)
DIRECT_INDICATOR = morph_pipeline(DIRECT_INDICATORS)
PERSON_DIRECT = or_(rule(DIRECT_INDICATOR, NAME),
                    rule(NAME, DIRECT_INDICATOR)
                     ).interpretation(Person)
parser_person_direct = Parser(PERSON_DIRECT)

## Загрузка данных

In [4]:
df = pd.read_csv('test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,manager,Ага


## Требования к менеджеру

In [5]:
role = 'manager'
df_manager = df.copy()
df_manager = df_manager[df_manager['role'] == role]
df_manager['greeting'] = df_manager.text.apply(is_indicator, indicators=GREETINGS)
df_manager['representation'] = df_manager.text.apply(is_indicator, indicators=REPRESENTATIONS)
df_manager['goodbye'] = df_manager.text.apply(is_indicator, indicators=GOODBYES)

### - менеджер поздоровался

In [6]:
df_manager[df_manager['greeting']]

,dlg_id,line_n,role,text,greeting,representation,goodbye
2,0,2,manager,Добрый день,True,False,False
109,1,0,manager,Да здравствуйте когда заканчивается,True,False,False
165,2,1,manager,Здравствуйте,True,False,False
249,3,0,manager,Добрый день,True,False,False
358,5,21,manager,Да да да здравствуйте анастасия меня слышно да...,True,False,False


### - менеджер представил себя

In [7]:
df_manager[df_manager['representation']]

,dlg_id,line_n,role,text,greeting,representation,goodbye


### - менеджер попрощался

In [8]:
df_manager[df_manager['goodbye']]

,dlg_id,line_n,role,text,greeting,representation,goodbye
301,3,52,manager,Да до свидания,False,False,True
336,4,34,manager,Угу да до свидания,False,False,True


### - имя менеджера

In [9]:
df_ner = df.copy()
df_ner['name'] = df_ner.apply(role_name, role=role, parser_direct=parser_person_direct, 
                              parser_indirect=parser_person_indirect, axis=1)
df_ner[df_ner['name'].notna()]

,dlg_id,line_n,role,text,name
250,3,1,client,Алло дмитрий добрый день,дмитрий
319,4,17,client,А вот а ну вот помните айдар вам не подходило ...,айдар
438,5,101,client,Вот смотрите вячеслав сейчас со мной я вам выш...,вячеслав


### - название компании

In [10]:
df_ner['company'] = df_ner.text.apply(company_recognize, parser=parser_company)
df_ner[df_ner['company'].notna()]

,dlg_id,line_n,role,text,name,company
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,None,диджитал бизнес
111,1,2,client,Меня зовут ангелина компания диджитал бизнес з...,None,диджитал бизнес
167,2,3,client,Меня зовут ангелина компания диджитал бизнес з...,None,диджитал бизнес
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,None,китобизнес


### - в каждом диалоге менеджер поздоровался и попрощвлся

In [11]:
greeting = df_manager.groupby('dlg_id')['greeting'].sum().astype(bool)
goodbye = df_manager.groupby('dlg_id')['goodbye'].sum().astype(bool)
representation = df_manager.groupby('dlg_id')['representation'].sum().astype(bool)

name = df_ner.groupby('dlg_id')['name'].apply(extract_values)
company = df_ner.groupby('dlg_id')['company'].apply(extract_values)

In [12]:
df_final = greeting.to_frame()
df_final = df_final.join(goodbye)
df_final = df_final.join(representation)
df_final = df_final.join(name)
df_final = df_final.join(company)
df_final['polite_person'] = df_final.apply(polite_person, axis=1)
df_final

,greeting,goodbye,representation,name,company,polite_person
dlg_id,,,,,,
0,True,False,False,None,диджитал бизнес,False
1,True,False,False,None,диджитал бизнес,False
2,True,False,False,None,диджитал бизнес,False
3,True,True,False,дмитрий,китобизнес,True
4,False,True,False,айдар,None,False
5,True,False,False,вячеслав,None,False
